In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [18]:
import data_util.preprocess 
importlib.reload(data_util.preprocess)
from data_util.preprocess import *



import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config

importlib.reload(data_util.batcher)
importlib.reload(model)
importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)

from data_util.batcher import *
from evaluate import *
from model import *
from train import *
from train_util import *
from data_util.data import *



In [3]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

In [4]:
!pwd

/Users/rowancassius/Desktop/capstone/LSTM_Summarizer


In [5]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

# fill nas
dat.fillna('', inplace=True)

# train/test split
np.random.seed(111)
dat = dat.sample(frac=1)
train_size = int(.8*dat.shape[0])
train_data = dat[:train_size]
test_data = dat[train_size:]


In [6]:
pd.set_option('display.max_colwidth', -1)

In [7]:
train_data.sample(10)

,#,Task,Context,TaskSentence,Summary,Labeler,NoRequestInContext,Urgent,NotRequest,Unsure/Discuss,RandomNumber
1166,5482,Please add Angela Davis to all e-mails regarding Project Tahiti.,,Please add Angela Davis to all e-mails regarding Project Tahiti.,Add Angela Davis to emails regarding Project Tahiti,Natalie,1,0,0,,0.504486
93,94,Susan: Please change the signature block as set forth below and I'll,,Susan: Please change the signature block as set forth below and I'll review.,Change signature block,Rowan,,,,,0.555939
915,5231,"comments, please call Jack Cashin at 202/ 5085-499 .","The FERC staff report on western markets and the causes of the Summer 2000 Price Abormalities, entitled, Part I of Staff Report on U.S.. Bulk Power Markets, is available at the following website: http://www.FERC.Fed.US/Electric/BulkPower.htm.","If you have any questions or comments, please call Jack Cashin at 202/ 5085-499 .",Call Jack Cashin at 202-5085-499 with questions,Natalie,1,0,0,,0.619421
782,5098,Could you get me a copy please?,"I'm going over the Park & Loan discounts on CNG for May.. I need to see the invoice to compare what we were billed to what CNG is showing on the discount letter.. Gloria, Terry does not have the invoice.",Could you get me a copy please?,Give SENDER a copy of the invoice for CNG,Natalie,1,0,0,,0.420132
162,163,Review the space plans and give your comments,"Let's schedule a meeting for Mark Taylor, you and me with the consultant who we already had conversations with.. We can formulate our response.. Mark. Pursuant to your request, I called Deborah Bubenko regarding what information she needs from you regarding space planning for the new building.. Deborah had Kim Kaase extension 35349 return my call.. Listed below are the next steps:",Review the space plans and give your comments,Review space plans,Rowan,1,,,,0.104132
917,5233,"If interested, please submit app to yeah@haas by","@ Promotes students' academic and personal development in a safe, nurturing environment.. Benefits for Mentors:. -Enhance the financial education and academic preparedness of under-served students;. -Get a crash course in stock market basics, investment strategies, macroecon issues, etc.. ;. -Earn a unit of academic credit for volunteering with Haas' youth outreach program.. font face Times New Roman, Times size 4 color #0000FF div align center Attached are the Saturday Session calendar and the Mentor/Coach application.&nbsp;","If interested, please submit app to yeah@haas",Submit app to yeah@haas,Natalie,1,0,1,,0.192327
1058,5374,Please contact me if you can inform me on any of the following subjects.,I am working on a report aimed at Energy Marketers that will examine Natural Gas Storage: Effects on Energy Trading.. I am looking for people who can talk about the importance of this subject to power marketers.,Please contact me if you can inform me on any of the following subjects.,Contact SENDER with information about Natural Gas Storage: Effects on Energy Trading report,Natalie,1,0,0,,0.483459
707,3474,"PS. Sean, please forward this to Richard.",,Please plan to meet on Wed. see below to discuss the Deal Profitability Analysis with Kim Kouri.,Plan to meet on Wednesday.,George,,,,,0.186233
1228,5544,"Vince, could you follow this up and","Hi Vince, I talked to Tom Gros today and metioned our desire to estabilish a strategic relationship with Nick Bambos and his research effort.. Tom is very excited about the idea and was willing to cut the check.. I suggested that we have more of a strategic relationship in mine Enron / Stanford and he agreed to that as well.. Tom mentioned that Greg Whalley was interested in estabilishing such a relationship.","Vince, could you follow this up and ground this?",Follow up on strategic relationship with Greg Whalley,Natalie,1,0,0,,0.344668
507,508,Please fill out the,The Raptor I structure requires that certian information be included with the confirmation of derivative transactions with Talon.,Please fill out the a

##  Vocabulary Curation

In [8]:
vocab_sents = [vocab_process_text(s) for s in 
               train_data.Context.values + train_data.TaskSentence.values]

In [9]:
# fit tokenizer
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(vocab_sents + list(ENT_TAGS))
assert all([t in tokenizer.word_index for t in ENT_TAGS])

In [10]:
words2vocabfile(tokenizer.word_index.keys(), os.path.join(config.log_root, 'data/vocab.txt'))

In [11]:
vocab = Vocab.from_vocab_file(os.path.join(config.log_root, 'data/vocab.txt'))

Finished constructing vocabulary of 4178 total words. Last word added: a


In [25]:
i= 1060
context = train_data.loc[i]['Context']
task    = train_data.loc[i]['TaskSentence']
summary = train_data.loc[i]['Summary']

In [26]:
task

"Please verify if this is some other c/p's deal."

In [27]:
task

"Please verify if this is some other c/p's deal."

In [28]:
# TODO write a unit test for these
vocab_process_text(task)

"please verify if this is some other c / p 's deal ."

In [29]:
article_process_text(task)

"Please verify if this is some other c/p's deal."

In [30]:
summary_process_text(task)

"please verify if this is some other c / p 's deal"

In [501]:
import data_util.batcher
importlib.reload(data_util.batcher)
from data_util.batcher import *

In [502]:
ex = TaskExample(context, task, summary, vocab)

In [503]:
len(ex.enc_input)

92

In [504]:
# list(zip(ex.enc_input_extend_vocab, ex.enc_input))

In [505]:
ex.dec_input

[2, 0, 13, 64, 3481, 3615, 3932, 3944, 0, 331]

In [506]:
ex.article_oovs

['the Office of the', 'PhoneMail Box', '713-8537-294', ' ', '20 or']

In [507]:
ex.summary_tokens

['Submit',
 'name',
 'and',
 'number',
 'with',
 'message',
 'to',
 'Office of the',
 'chairman']

In [499]:
ex.sum_ids_extend_vocab

[0, 13, 64, 3481, 3615, 3932, 3944, 4178, 331]

In [500]:
ex.dec_input

[2, 0, 13, 64, 3481, 3615, 3932, 3944, 0, 331]

In [510]:
ex.summary.ents

(Submit, Office of the)

In [513]:
text.split(' ', 1)

['Carl,',
 'my phone was lost. Would you ask SENDER to look into another and spam my website: www.car.com. Additional this person might have a lead: Joe Anderson.']